# Slot Filling

In [1]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#change current working directory
%cd "/content/drive/Shareddrives/NLI"

/content/drive/Shareddrives/NLI


In [3]:
#Print path and content of the current directory
!pwd
!ls

/content/drive/Shareddrives/NLI
 1_dialogue_act_best_2.ipynb	   2_slot_fitting.ipynb		    logs
 1_dialogue_act_best.ipynb	  'Còpia de 2_slot_fitting.ipynb'   model
 1_dialogue_act_prediction.ipynb   dataset.hf			    results


In [4]:
!pip install transformers
!pip install seqeval
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 566.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6f2a761f1a9f1f3395a0e8b8b2109e403d7427ff59fc7e2b305796c7e1ccb28e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00


In [5]:
!python3 -m spacy download en_core_web_lg

2023-11-28 17:15:51.310891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 17:15:51.310957: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 17:15:51.311004: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 17:15:51.324053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 17:15:53.852907: W tensorflow/compiler/

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertTokenizerFast, BertConfig, BertForTokenClassification
from seqeval.metrics import classification_report
from datasets import load_dataset

import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from nltk import word_tokenize
from tqdm import tqdm
import re

nltk.download('punkt')

import spacy

nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
#DATASET_FILEPATH = './events/attack.txt'
TRANSFORMER_MODEL_NAME = 'bert-large-uncased'

TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05

In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
def preprocess_split(dataset, split):
    df = dataset[split].to_pandas()
    new_df = pd.DataFrame(columns = df.columns)
    for i in range(len(df)):
        # Taken from notebook, to know which lines to skip
        row = df.loc[i]
        if not any(set(row.turns['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(row.turns['utterance'])):
            continue

        new_df.loc[len(new_df)] = row
        # new_df.loc[len(new_df) - 1]['services'] = process_service_list(new_df.loc[len(new_df) - 1]['services'])
        # for i, frame_service in [frame['service'] for frame in df.loc[i].turns['frames']]:
            # df.loc[i].turns['frames']
    return new_df

In [10]:
def process_service_list(service_list):
    services = set()
    if len(service_list) == 0:
        services.add('other')
    for service in service_list:
        if service == 'restaurant':
            services.add('restaurant')
        elif service == 'hotel':
            services.add('hotel')
        else:
            services.add('other')
        if len(services) == 3:
            break
    return list(services)

In [11]:
#dataset = load_dataset('multi_woz_v22')

from datasets import DatasetDict
dataset = DatasetDict.load_from_disk("dataset.hf")

train = preprocess_split(dataset, 'train')
val = preprocess_split(dataset, 'validation')
test = preprocess_split(dataset, 'test')

In [12]:
ignored_tags = []

In [13]:
def extract_token_bio_tags(dataset):
    utterances = [[]]
    count = 0

    for i in tqdm(range(len(dataset))):
        turns = dataset.loc[i].turns
        for utterance, speaker, dialogue_act, frames in zip(turns['utterance'], turns['speaker'], turns['dialogue_acts'], turns['frames']):
            if speaker != 0:
                continue
            if 'other' in process_service_list(frames['service']):
                continue

            span_info = dialogue_act['span_info']
            act_slot_names = span_info['act_slot_name']
            act_slot_values = span_info['act_slot_value']
            span_starts = span_info['span_start']
            span_ends = span_info['span_end']
            slots = {slot_name : {'start': start, 'end': end} for slot_name, start, end in zip(act_slot_names, span_starts, span_ends)}

            #Preprocess

            last_tag = 0
            utterances.append([])
            for slot_name in slots:
                slot_start, slot_end = slots[slot_name]['start'], slots[slot_name]['end']
                tokens = nlp(" ".join(utterance[slot_start:slot_end].lower().split()))
                #tokens = re.sub("[^a-zA-Z0-9]", " ", utterance[slot_start:slot_end]).lower().split()
                for word in nlp(" ".join(utterance[last_tag:slot_start].lower().split())):
                #for word in re.sub("[^a-zA-Z0-9]", " ", utterance[last_tag:slot_start]).lower().split():
                  utterances[count].append((str(word), "O"))

                for j, token in enumerate(tokens):
                    bio_type = 'B-' if j == 0 else 'I-'
                    if bio_type + slot_name not in ignored_tags:
                        utterances[count].append((str(token), bio_type + slot_name))
                last_tag = slot_end

            #for word in re.sub("[^a-zA-Z0-9]", " ", utterance[last_tag:]).lower().split():
            for word in nlp(" ".join(utterance[last_tag:].lower().split())):
                  utterances[count].append((str(word), "O"))
            count = count+1


    return utterances

In [14]:
# building X, y
train_word_tag = extract_token_bio_tags(train)
valid_word_tag = extract_token_bio_tags(val)
test_word_tag = extract_token_bio_tags(test)

print("\nSize train_sentencs: ", len(train_word_tag))
print("Size valid_sentences: ", len(valid_word_tag))
print("Size test_sentences: ", len(test_word_tag))



100%|██████████| 745/745 [00:59<00:00, 12.62it/s]


Size train_sentencs:  23542
Size valid_sentences:  2668
Size test_sentences:  2619


In [15]:
from collections import Counter
print("Number of utterances:", len(train_word_tag))
print("Frequency")
counter = sorted(Counter([tag for sentence in train_word_tag for _, tag in sentence]).items(), key=lambda x: x[1], reverse=True)
for tag, val in counter:
    print(f'{tag:20} {val}')

print("\nEach tag appears in number of documents")
docs = {}
for i, sentence in enumerate(train_word_tag):
    for _, tag in sentence:
        if tag not in docs:
            docs[(i, tag)] = 1
        else:
            docs[(i, tag)] += 1
counter = Counter([tag for i, tag in docs])
for tag in counter:
  print(f'{tag:20} {counter[tag]}')


Number of utterances: 23542
Frequency
O                    296492
B-area               4324
B-pricerange         4301
B-food               3805
B-bookpeople         3223
B-bookday            3208
B-bookstay           2306
I-name               2301
B-type               2150
B-stars              1841
B-name               1787
B-booktime           1561
I-area               826
I-food               649
I-pricerange         443
I-bookpeople         226
I-bookday            150
I-type               60
I-booktime           50
I-bookstay           2

Each tag appears in number of documents
O                    23532
B-area               4324
B-pricerange         4301
B-type               2150
B-bookstay           2306
B-bookpeople         3223
B-bookday            3208
I-pricerange         219
B-name               1787
I-name               1406
B-food               3805
B-booktime           1561
I-food               492
B-stars              1841
I-area               517
I-bookday            15

In [16]:
all_words = set([word for sentence in train_word_tag for word, _ in sentence])

max_seq_len = max([len(x) for x in train_word_tag])
print("max_seq_leng:", max_seq_len)

n_words = len(all_words)
print("Number of words:", n_words)
print("Words: ", all_words)

# As the BertTokenizer uses wordpiece, the number of tokens is higher than the actual word count, so we set the max_seq_len to 128
max_seq_len = 128

max_seq_leng: 42
Number of words: 2424
Words:  {'vue', 'bloomsbury', 'end', 'host', 'limited', 'reiterate', 'normal', 'joining', 'offer', 'and2', 'two', 'judgment', 'at11:00', 'awesome', '/5', 'physician', 'recommend', 'unable', 'pasta', 'postcodes', 'go', 'areas', 'meal', 'since', 'disregard', 'conference', 'these', 'authentic', 'idea', 'restairant', 'yourself', 'okay', 'tag', 'concerened', 'self', 'lives', '-sunday', 'gandhi', 'aat', 'fen', 'see', 'checked', 'grab', 'centrally', 'any', 'itshould', 'work.i', '12:15', 'tr6838', 'condo', 'spending', 'nevermind', 'costly', 'stay/', 'others', 'arrangements', 'zero', 'uno', 'sure', '200', 'previous', 'alexander', '6', 'again', 'lensfield', 'galleries', 'fit', 'rethinking', 'inspire', 'deal', 'response', 'addresss', 'never', 'pricier', 'adjoining', '-themed', 'pathological', 'teusday', 'wi', 'celebrating', 'ugly', 'east', '16:15', 'french', 'colleagues', 'stated', 'mondern', 'win', 'do', 'swear', '451', 'broughton', 'finches', 'turkish', 'r

In [17]:
all_tags = set([tag for sentence in train_word_tag for _, tag in sentence])
num_tags = len(all_tags)
print("Number of tags:", num_tags)
print("Tags:", all_tags)

Number of tags: 20
Tags: {'B-booktime', 'I-booktime', 'I-bookpeople', 'I-type', 'B-food', 'O', 'B-stars', 'B-bookpeople', 'B-area', 'I-bookstay', 'I-area', 'B-type', 'I-name', 'B-bookstay', 'B-name', 'I-food', 'I-pricerange', 'B-pricerange', 'B-bookday', 'I-bookday'}


In [18]:
tag2index = {tag: i for i, tag in enumerate(all_tags)}
index2tag = {i: tag for tag, i in tag2index.items()}

In [19]:
tag2index

{'B-booktime': 0,
 'I-booktime': 1,
 'I-bookpeople': 2,
 'I-type': 3,
 'B-food': 4,
 'O': 5,
 'B-stars': 6,
 'B-bookpeople': 7,
 'B-area': 8,
 'I-bookstay': 9,
 'I-area': 10,
 'B-type': 11,
 'I-name': 12,
 'B-bookstay': 13,
 'B-name': 14,
 'I-food': 15,
 'I-pricerange': 16,
 'B-pricerange': 17,
 'B-bookday': 18,
 'I-bookday': 19}

In [20]:
index2tag

{0: 'B-booktime',
 1: 'I-booktime',
 2: 'I-bookpeople',
 3: 'I-type',
 4: 'B-food',
 5: 'O',
 6: 'B-stars',
 7: 'B-bookpeople',
 8: 'B-area',
 9: 'I-bookstay',
 10: 'I-area',
 11: 'B-type',
 12: 'I-name',
 13: 'B-bookstay',
 14: 'B-name',
 15: 'I-food',
 16: 'I-pricerange',
 17: 'B-pricerange',
 18: 'B-bookday',
 19: 'I-bookday'}

In [21]:
# Initialize Bert tokenizer
tokenizer = BertTokenizerFast.from_pretrained(TRANSFORMER_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [22]:
class dataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
        self.len = len(X)
        self.X = X
        self.y = y
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.X[index]
        word_labels = self.y[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [tag2index[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

In [23]:
X_train = [[word for word, _ in sentence] for sentence in train_word_tag]
y_train = [[tag for _, tag in sentence] for sentence in train_word_tag]

X_valid = [[word for word, _ in sentence] for sentence in valid_word_tag]
y_valid = [[tag for _, tag in sentence] for sentence in valid_word_tag]

X_test = [[word for word, _ in sentence] for sentence in test_word_tag]
y_test = [[tag for _, tag in sentence] for sentence in test_word_tag]

training_set = dataset(X_train, y_train, tokenizer, max_seq_len)
validation_set = dataset(X_valid, y_valid, tokenizer, max_seq_len)
testing_set = dataset(X_test, y_test, tokenizer, max_seq_len)

In [24]:
training_set[0]

{'input_ids': tensor([  101,  1045,  2342,  1037,  2173,  2000, 11586,  2063,  1999,  1996,
          2415,  2008,  1055,  6450,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [25]:
index = 1
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[index]["input_ids"]), training_set[index]["labels"]):
  print(f'{token:10} {label:3} ')

[CLS]      -100 
any          5 
sort         5 
of           5 
food         5 
would        5 
be           5 
fine         5 
,            5 
as           5 
long         5 
as           5 
it           5 
is           5 
a            5 
bit          5 
expensive    5 
.            5 
could        5 
i            5 
get          5 
the          5 
phone        5 
number       5 
for          5 
your         5 
recommendation   5 
?            5 
[SEP]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PAD]      -100 
[PA

In [26]:
training_loader = DataLoader(training_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=VALID_BATCH_SIZE, shuffle=False)
testing_loader = DataLoader(testing_set, batch_size=VALID_BATCH_SIZE, shuffle=False)

In [27]:
model = BertForTokenClassification.from_pretrained(TRANSFORMER_MODEL_NAME, num_labels=len(tag2index))
model.to(device)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

In [28]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.9810, device='cuda:0', grad_fn=<NllLossBackward0>)

In [29]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 20])

In [30]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [31]:
def valid(model, data_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            # if idx % 100==0:
            #     loss_step = eval_loss/nb_eval_steps
            #     print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [index2tag[id.item()] for id in eval_labels]
    predictions = [index2tag[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps

    # print(f"Validation : loss {eval_loss:.2f} accuracy {eval_accuracy:.2f}")

    return labels, predictions, eval_loss, eval_accuracy

In [32]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    with tqdm(training_loader, 'batch') as tepoch:
        for idx, batch in enumerate(tepoch):
            tepoch.set_description(f'Epoch {epoch}')

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss, tr_logits = outputs[0], outputs[1]
            tr_loss += loss.item()

            nb_tr_steps += 1
            nb_tr_examples += labels.size(0)

            # if idx % 100==0:
            #     loss_step = tr_loss/nb_tr_steps
            #     print(f"Training loss per 100 training steps: {loss_step:.2f}")

            # compute training accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
            #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            tr_labels.extend(labels)
            tr_preds.extend(predictions)

            tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            tr_accuracy += tmp_tr_accuracy

            # gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=10
            )

            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=tr_loss / nb_tr_steps, accuracy=tr_accuracy / nb_tr_steps)

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps

    _, _ , val_loss, val_acc = valid(model, validation_loader)

    return epoch_loss, tr_accuracy, val_loss, val_acc


In [ ]:
history = {'train_loss':[], 'train_accuracy':[], 'val_loss':[], 'val_accuracy':[], 'epoch':[]}
for epoch in range(EPOCHS):
    train_loss, train_acc, val_loss, val_acc = train(epoch)

    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    history['epoch'].append(epoch+1)


Epoch 2:  39%|███▉      | 1152/2943 [12:41<19:46,  1.51it/s, accuracy=0.984, loss=0.0459]

In [ ]:
plt.figure(1)
plt.plot(history[f'train_loss'])
plt.plot(history[f'val_loss'])
plt.title(f'model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show

plt.figure(2)
plt.plot(history[f'train_accuracy'])
plt.plot(history[f'val_accuracy'])
plt.title(f'model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show

In [ ]:
labels, predictions, loss, accuracy = valid(model, testing_loader)
print(f"Validation : loss {loss:.2f} accuracy {accuracy:.2f}")

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

# for source, pred in zip(labels, predictions):
#   print(f'{source:20} {pred}')

In [ ]:
model.save_pretrained('./model/task2_nlp')